## Notebook for training ML models on dataset for Open-Source Reservoir (CMG simulations)

**Important:** running this notebook takes a long time (in the order of 24 hours). The idea here is to run cross-validation implemented in this notebooks once to find best hyperparameters and then we can use a different notebook to train production models which will take much less time. 

-------
Code in this notebook trains multiple models and performs k-fold cross-validation. 
Exploration of model/training parameters includes:
* varying degree of interpolating polygons for studied timeseries
* varying the numbers of neurons in the NN hidden layers (networks with two hidden layers are studied)
* varying the amount of training (number of epochs)

MAPE and MAE error measures (presented in the error dataframe) are averaged across both validation cases and k-folds.  

This exporation is repeated for all quantities of interest (pressures and temperatures for producer wells).

Handing of the analysis results is implemented in such a way where old results are red in and new results are added to the growing dataframe; 
after each interation, the up-to-date dataframe is saved to disk to prevent loss of results in situations where jupyter sessions are interrupted.

-------

*Written by: Dmitry Duplyakin (dmitry.duplyakin@nrel.gov) in collaboration with the National Renewable Energy Laboratories.*

*Full team: Dmitry Duplyakin, Koenraad F. Beckers, Drew L. Siler, Michael J. Martin, Henry E. Johnston*

### Necessary configuration

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import uuid
from datetime import datetime
import glob
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.metrics import mean_absolute_error
import math
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import KFold

loglevel = 'WARNING'
logging.basicConfig(level=os.environ.get("LOGLEVEL", loglevel))

# Import config file that is specific to CMG dataset
sys.path.append('../data/OpenSourceReservoir-CMG')
sys.path.append('../')

import config_cmg as config

#sys.path.append('..')
from reservoir.reservoir import Reservoir, ReservoirPredictionEnsemble

from polynomial import get_polynomial_func

%matplotlib inline
%config InlineBackend.figure_format='retina'

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

### Loading data (from Xcel files to Pandas dataframes)

In [2]:
cases_list = []

data_dir = "../data/OpenSourceReservoir-CMG/"
filename_pattern = "ML_BHS_JA_*_ML_BHS_JA.xlsx"

# This assumes file names like: ../data/TETRADG-60_cases/TETRADG_v2_<case #>.xlsx
cases = dict([(int(f.split("/")[-1].replace(".xlsx","").replace("ML_BHS_JA", "").replace("_","")),f) for f 
         in glob.glob(data_dir + filename_pattern)])

for case in sorted(cases):

    file = cases[case]
    print("Processing: case %d, file %s" % (case, file))
    
    config_for_case = config
    
    # Override some config setting for this particular vis/analysis
    config_for_case.flow_unit = "kg/day"
    config_for_case.timeseries_file = file

    cases_list.append(Reservoir(config_for_case, energy_calc=False))

Processing: case 0, file ../data/OpenSourceReservoir-CMG/ML_BHS_JA_00000_ML_BHS_JA.xlsx
Processing: case 1, file ../data/OpenSourceReservoir-CMG/ML_BHS_JA_00001_ML_BHS_JA.xlsx
Processing: case 2, file ../data/OpenSourceReservoir-CMG/ML_BHS_JA_00002_ML_BHS_JA.xlsx
Processing: case 3, file ../data/OpenSourceReservoir-CMG/ML_BHS_JA_00003_ML_BHS_JA.xlsx
Processing: case 4, file ../data/OpenSourceReservoir-CMG/ML_BHS_JA_00004_ML_BHS_JA.xlsx
Processing: case 5, file ../data/OpenSourceReservoir-CMG/ML_BHS_JA_00005_ML_BHS_JA.xlsx
Processing: case 6, file ../data/OpenSourceReservoir-CMG/ML_BHS_JA_00006_ML_BHS_JA.xlsx
Processing: case 7, file ../data/OpenSourceReservoir-CMG/ML_BHS_JA_00007_ML_BHS_JA.xlsx
Processing: case 8, file ../data/OpenSourceReservoir-CMG/ML_BHS_JA_00008_ML_BHS_JA.xlsx
Processing: case 9, file ../data/OpenSourceReservoir-CMG/ML_BHS_JA_00009_ML_BHS_JA.xlsx
Processing: case 10, file ../data/OpenSourceReservoir-CMG/ML_BHS_JA_00010_ML_BHS_JA.xlsx
Processing: case 11, file ../da

### Combine different scenarios into an ensemble

In [3]:
ens = ReservoirPredictionEnsemble(config, cases_list)    
ens.scale()

# To see individual scaled timeseries, do the following:
# ens[0].scaled_timeseries

### Model training and evaluation (with cross-validation)

Keep in mind that running the following code takes a **long time!** (unless `quick_test` is set to `True`)

In [4]:
# This cell: 
# 1) sets aside 10% of data for TESTING, and 2) performs k-fold cross-validation on the training data (90% of all data)

# Like mentioned at the top of the notebook, the code here is written to run "incrementally": 
# it will find saved results (in a file like: ../results/error_summaries/batch-OSR-CV-test.csv) if there are any,
# and continue running analysis for missing configurations.

##########################
# Setting key parameters #
##########################

batch_id = "OSR-CV-quick"

loss = "mae" # mae or mse
validation_split_ratio = 0.0 # Ratio of train set that will be treated as validation set

k = 10 # k for k-fold validation

plotting = False

res_dir = "../results/error_summaries/"
model_dir = "../models/"

save_to_disk = False

quick_test = True

if quick_test:
    quantity_list = ["pp1"]
    degree_list = [4,5,6]
    nn_list = [[12, 6]]
    n_epochs_list = [10]   
else:
    # This configuration will require a lot of compute time!
    quantity_list = ["pp1", "pp2", "pp3", "pp4", "pp5", "pp6", "pt1", "pt2", "pt3", "pt4", "pt5", "pt6"]
    degree_list = [4,5,6]
    nn_list = [[12, 6],
               [12, 12],
               [16, 8],
               [16, 16],
               [24, 12],
               [24, 24],
               [32, 16],
               [32, 32]]
    n_epochs_list = [250, 500, 1000]   

# New results will be *appended* to this file
dest_file = os.path.join(res_dir, "batch-%s.csv" % (str(batch_id)))

required_results_df_columns = ["timestamp", "train_idx", "quantity", "degree", "nn", 
                               "n_epochs", "loss", "k", "mape_list", "mae_list", "mape_avg", "mae_avg"]                        
if os.path.exists(dest_file):
    existing_results_df = pd.read_csv(dest_file)
else:
    # Empty dataframe with required columns
    existing_results_df = pd.DataFrame(columns = required_results_df_columns)
    
##################################
# All key parameters are set now #
##################################

##################################
# Routine for model fitting      #
##################################

def fit_custom_model(X_train, Y_train, nn, loss, n_epochs):
    
    assert len(nn) == 2, "nn: should be a list with *two* layer sizes"
    nn1, nn2 = nn
    
    # initialize model
    model = Sequential()

    # add 1st layer
    model.add(Dense(
        units=nn1,
        input_dim=X_train.shape[1],
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='tanh') 
    )
    # add 2nd layer
    model.add(
        Dense(
            units=nn2,
            input_dim=nn1,
            kernel_initializer='glorot_uniform',
            bias_initializer='zeros',
            activation='tanh')
        )
    # add output layer
    model.add(
        Dense(
            units=Y_train.shape[1],
            input_dim=nn2,
            kernel_initializer='glorot_uniform',
            bias_initializer='zeros',
            activation=None)
        )

    # define SGD optimizer
    sgd_optimizer = SGD(
        lr=0.001, decay=1e-7, momentum=0.9
    )
    
    # compile model
    model.compile(
        optimizer=sgd_optimizer,
        loss=loss
    )

    X_train_converted=np.asarray(X_train).astype(np.float32)
    Y_train_converted=np.asarray(Y_train).astype(np.float32)

    #print(model.summary())

    #train model; validation_split=0.0 because CV is already handled outside of this training routine
    training_history = model.fit(
        X_train_converted, Y_train_converted,
        batch_size=1, 
        epochs=n_epochs,
        verbose=0, 
        validation_split=0.0
    )
    return model, training_history

#################################### 
# End of Routine for model fitting #
####################################

# Set aside indices for TESTING (after cross validation)
all_idx = np.array(range(ens.count))
kfold = KFold(10, True, 1) # 10-fold results in 10% of all indices (101) being exactly 11 cases
train, test = next(kfold.split(all_idx))
train_idx, test_idx = all_idx[train], all_idx[test]
print('train: %s, test: %s' % (train_idx, test_idx))

# Form a dataframe with all controlled parameter values
m_column_list = ["im1", "im2", "im3", "im4"]
injector_df = pd.DataFrame(index=range(ens.count), columns=m_column_list)
for i in range(ens.count):
    df = ens[i].scaled_timeseries[m_column_list]
    df = df[df.index >= pd.to_datetime('2020-01-01')]

    for c in m_column_list:
        # Important: this characterizes a timeseries by its most common values; different methods can be used here
        # This works well for the step function
        injector_df.at[i, c] = df[c].value_counts().index[0]

m_column_list = ["pm1", "pm2", "pm3", "pm4", "pm5", "pm6"]
producer_df = pd.DataFrame(index=range(ens.count), columns=m_column_list)
for i in range(ens.count):
    df = ens[i].scaled_timeseries[m_column_list]
    df = df[df.index >= pd.to_datetime('2020-01-01')]

    for c in m_column_list:
        # Important: this characterizes a timeseries by its most common values; different methods can be used here
        # This works well for the step function
        producer_df.at[i, c] = df[c].value_counts().index[0]

all_wells_df = injector_df.join(producer_df)
display(all_wells_df)

for degree in degree_list:
    print("degree:", degree)
    poly = get_polynomial_func(degree=degree)
    
    for n_epochs in n_epochs_list: 
        print("n_epochs:", n_epochs)
        for quantity in quantity_list:
            print("Quantity:", quantity)
            for nn in nn_list:
                print("nn:", nn)

                if len(existing_results_df) > 0:
                    matching_existing_results = existing_results_df[(existing_results_df["degree"] == degree) &
                                                                    (existing_results_df["n_epochs"] == n_epochs) &
                                                                    (existing_results_df["quantity"] == quantity) &
                                                                    (existing_results_df["nn"] == str(nn))]
                    if len(matching_existing_results) > 0:
                        print("\nSkipping (found existing results).")
                        continue
                         
                if quantity in ["pp1", "pp2", "pp3", "pp4", "pp5", "pp6"]:
                    scaler = ens.common_pres_scaler
                elif quantity in ["pt1", "pt2", "pt3", "pt4", "pt5", "pt6"]:
                    scaler = ens.common_temp_scaler

                t_mapper = ens.shared_scaled_time_index(start_at='2020-01-01')
                #t_mapper = {k: v+1.0 for k,v in t_mapper.items()}

                r2_vector = []
                rmse_vector = []

                ydata_df = pd.DataFrame(index=sorted(t_mapper.values()), columns=range(ens.count))
                yhat_df = pd.DataFrame(index=sorted(t_mapper.values()), columns=range(ens.count))

                coeff_df, _, _ = ens.get_curve_approximations(quantity, poly)

                #display(coeff_df)

                mape_list, mae_list = [], []

                # Further split train set into actual train and validate subset within k-fold validation routine
                kfold_within_train = KFold(k, True, 1) # k-fold CV
                for kfold_id, (actual_train, actual_val) in enumerate(kfold_within_train.split(train_idx)):
                    actual_train_idx, actual_val_idx = train_idx[actual_train], train_idx[actual_val]
                    #print('actual train: %s, actual val: %s' % (actual_train_idx, actual_val_idx))

                    #print("Input for Curve ML training:")
                    X_train = all_wells_df.loc[actual_train_idx]
                    #display(X_train)

                    #print("Output for Curve ML training:")
                    Y_train = coeff_df.loc[actual_train_idx]
                    #display(Y_train)

                    X_val = all_wells_df.loc[actual_val_idx]
                    Y_val = coeff_df.loc[actual_val_idx]

                    model, training_history = fit_custom_model(X_train, Y_train, nn, loss, n_epochs)

                    # Random UUID
                    model_uuid = uuid.uuid4()
                    if save_to_disk:
                        model.save(os.path.join(model_dir, str(model_uuid)))

                    X_val=np.asarray(X_val).astype(np.float32)
                    Y_val=np.asarray(Y_val).astype(np.float32)

                    # Predicted coefficients
                    Y_val_pred = model.predict(X_val, verbose=0)

                    xdata = np.linspace(0, 1.0, 50) # point along scaled time dimension

                    for coeff_true, coeff_pred, val_case_id in zip(Y_val, Y_val_pred, actual_val_idx):
                        one_traj = ens[val_case_id].scaled_timeseries[quantity]
                        one_traj = one_traj[one_traj.index >= pd.to_datetime('2020-01-01')]
                        xdata_for_true_data = np.array(one_traj.index.map(t_mapper))
                        ydata_not_fitted = one_traj.values

                        ydata_true = poly(xdata, *coeff_true)
                        ydata_pred = poly(xdata, *coeff_pred)
                        ydata_pred_for_error_est = poly(xdata_for_true_data, *coeff_pred)

                        # Perform error analysis on unscaled data
                        ydata_not_fitted_unscaled = scaler.inverse_transform(ydata_not_fitted.reshape(-1, 1)).reshape(-1,)
                        ydata_pred_for_error_est_unscaled = \
                            scaler.inverse_transform(ydata_pred_for_error_est.reshape(-1, 1)).reshape(-1,)
                        mape = mean_absolute_percentage_error(ydata_not_fitted_unscaled, ydata_pred_for_error_est_unscaled)
                        mae = mean_absolute_error(ydata_not_fitted_unscaled, ydata_pred_for_error_est_unscaled)

                        mape_list.append(mape)
                        mae_list.append(mae)

                now = datetime.now()
                new_results_df = pd.DataFrame(columns = required_results_df_columns)
                new_results_df.loc[len(new_results_df)] = [
                    now, 
                    train_idx,
                    quantity, degree, nn, n_epochs, loss, 
                    k,
                    mape_list, mae_list, np.array(mape_list).mean(), np.array(mae_list).mean()]
                
                # Combine existing and new results
                if len(existing_results_df) > 0:
                    existing_results_df = pd.concat([existing_results_df, new_results_df])
                else: 
                    existing_results_df = new_results_df
                
                existing_results_df.to_csv(dest_file, index=False)
                print("\nUpdated file: %s (curr. length: %d)" % (dest_file, len(existing_results_df)))
                
display(existing_results_df)
print("Done!")

train: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 37 38 39 40 41 42 43 44 45 46 47 48 49
 50 51 53 54 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
 76 77 79 80 81 84 86 87 88 89 90 91 92 93 96 97 98 99], test: [ 17  36  52  55  78  82  83  85  94  95 100]


,im1,im2,im3,im4,pm1,pm2,pm3,pm4,pm5,pm6
0,0.43091,0.639029,1.38781,0.681051,0.950629,0.460185,0.496246,0.493607,0.970607,0.449299
1,0.43091,0.479272,1.73477,0.510788,0.950629,0.690278,0.496246,0.370205,0.485303,0.224649
2,0.43091,0.319515,1.38781,0.681051,1.18829,0.230093,0.620308,0.493607,0.485303,0.22465
3,0.323182,0.798787,1.73477,0.340525,1.18829,0.345139,0.620308,0.370205,0.970607,0.561624
4,0.646365,0.479272,1.04086,1.02158,1.18829,0.575232,0.248123,0.617009,1.21326,0.336975
...,...,...,...,...,...,...,...,...,...,...
96,0.215455,0.639029,1.38781,0.340525,0.475314,0.575232,0.744369,0.246803,0.970607,0.4493
97,0.646365,0.319515,1.38781,0.510788,0.950629,0.690278,0.496246,0.246804,1.21326,0.561624
98,0.646365,0.639029,1.38781,0.340525,0.475314,0.230093,0.744369,0.370205,1.21326,0.22465
99,0.215455,0.958544,1.73477,0.340525,0.712971,0.345139,0.620308,0.246803,0.970607,0.561624


degree: 4
n_epochs: 10
Quantity: pp1
nn: [12, 6]


2021-12-23 19:39:46.864641: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2021-12-23 19:39:46.954138: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2300000000 Hz
2021-12-23 19:39:46.955193: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x556507bfe370 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-12-23 19:39:46.955830: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-12-23 19:39:46.956108: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.



Updated file: ../results/error_summaries/batch-OSR-CV-quick.csv (curr. length: 1)
degree: 5
n_epochs: 10
Quantity: pp1
nn: [12, 6]

Updated file: ../results/error_summaries/batch-OSR-CV-quick.csv (curr. length: 2)
degree: 6
n_epochs: 10
Quantity: pp1
nn: [12, 6]

Updated file: ../results/error_summaries/batch-OSR-CV-quick.csv (curr. length: 3)


,timestamp,train_idx,quantity,degree,nn,n_epochs,loss,k,mape_list,mae_list,mape_avg,mae_avg
0,2021-12-23 19:41:06.860257,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",pp1,4,"[12, 6]",10,mae,10,"[14.331176846576543, 3.40770548520133, 2.36013...","[2649.789763556826, 674.6109988724106, 553.434...",8.977256,1856.447272
0,2021-12-23 19:42:35.050184,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",pp1,5,"[12, 6]",10,mae,10,"[27.456637417699564, 14.983467066400676, 13.11...","[5044.152524204159, 2943.330869447975, 3098.16...",10.350087,2147.584989
0,2021-12-23 19:43:52.975104,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",pp1,6,"[12, 6]",10,mae,10,"[13.573262544441574, 12.339955892277068, 3.660...","[2514.630590167876, 2430.177386765585, 862.611...",8.941384,1850.860536


Done!
